# Add upstream, downstream and basin information to the dataframe

* Purpose of script: add contextual data to the dataframe. The script will sum the volumetric information of all upstream, downstream and basin parameters of the dataframe. The script was revised on 12 October 2017 to restrict negative runoff.
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20170915

In [1]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2017M10D12 UTC 15:37


'3.5.4 |Continuum Analytics, Inc.| (default, Aug 14 2017, 13:26:58) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
INPUT_VERSION = 18
OUTPUT_VERSION = 1

SCRIPT_NAME = "Y2017M09D15_RH_Add_Basin_Data_V02"

S3_INPUT_PATH_EE  = "s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/%s/output/" %(SCRIPT_NAME)

S3_INPUT_PATH_HYDROBASINS = "s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

EC2_INPUT_PATH = "/volumes/data/%s/input" %(SCRIPT_NAME)
EC2_OUTPUT_PATH = "/volumes/data/%s/output" %(SCRIPT_NAME)

INPUT_FILENAME_EE =  "mergedZonalStatsEE_V%0.2d.pkl" %(INPUT_VERSION)
INPUT_FILENAME_HYDROBASINS =  "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl"

OUTPUT_FILENAME = "Y2017M09D15_RH_Add_Basin_Data_V%0.2d" %(OUTPUT_VERSION)

Note: There are two polygons with the same PFAF_ID (353020). This is caused by the fact that both poygons would otherwise cross the 180 degree meridian

In [3]:
!rm -r {EC2_INPUT_PATH} 
!rm -r {EC2_OUTPUT_PATH} 

In [4]:
!mkdir -p {EC2_INPUT_PATH} 
!mkdir -p {EC2_OUTPUT_PATH} 

In [5]:
!aws s3 cp {S3_INPUT_PATH_EE} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V18.pkl to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V02/input/mergedZonalStatsEE_V18.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V18.csv to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V02/input/mergedZonalStatsEE_V18.csv


In [6]:
!aws s3 cp {S3_INPUT_PATH_HYDROBASINS} {EC2_INPUT_PATH} --recursive --exclude "*" --include "*.pkl"

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V02/input/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl


In [7]:
import os
import pandas as pd
import multiprocessing as mp
import pickle
import numpy as np
import itertools
import logging
import pprint
import ast

In [8]:
inputLocationEE = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_EE)
inputLocationHydroBasins = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_HYDROBASINS)

In [9]:
df_ee = pd.read_pickle(inputLocationEE)

In [10]:
df_ee.index.names = ['PFAF_ID']

In [11]:
df_HydroBasins = pd.read_pickle(inputLocationHydroBasins)


In [12]:
df_complete = df_HydroBasins.merge(df_ee,how="left",left_index=True, right_index=True)

Note: There are two polygons with the same PFAF_ID (353020). This is caused by the fact that both poygons would otherwise cross the 180 degree meridian. 

In [13]:
df_complete = df_complete.drop_duplicates(subset='PFAF_ID', keep='first')

## Functions

In [14]:
def calculateTotalDemand(useType,temporalResolution,year,month):
    # This function will add Dom Ind IrrLinear and Livestock of all basins in the input list
    
    if temporalResolution == "year":
        keyTotal = "total_volume_Tot%s_%s_Y%0.4d" %(useType,temporalResolution,year)
    else:
        keyTotal = "total_volume_Tot%s_%s_Y%0.4dM%0.2d" %(useType,temporalResolution,year,month)
    
    # Create Column with zeros
    dfDemand[keyTotal] = 0
    for demandType in demandTypes:
        if demandType == "IrrLinear" and temporalResolution == "year":
            key = "total_volume_%s%s_%sY%0.4d" %(demandType,useType,temporalResolution,year)
        else:
            key = "total_volume_%s%s_%sY%0.4dM%0.2d" %(demandType,useType,temporalResolution,year,month)
        dfDemand[keyTotal] = dfDemand[keyTotal] + df_complete[key]
    return dfDemand   





# This functions can take only one argument because I map them over the pooler.
def addUpstream2(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    df_out = pd.DataFrame(index=df_part_temp2.index)
    
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        try:
            upstreamCatchments = df_part_temp2.loc[index, "Upstream_PFAF_IDs"]
            upstreamCatchments = ast.literal_eval(upstreamCatchments)
            df_upstream = df_full_temp[df_full_temp.index.isin(upstreamCatchments)]
            # selecting columns based on regular expression
            df_upstream = df_upstream.filter(regex=("^total_(area|volume_(P|I))"))
            df_upstream = df_upstream.add_prefix("upstream_")
            
            # added later (2017 10 11) to prevent negative runoff from upstream to propagate downstream. 
            # Update: Setting the sum to zero instead of the input runoff parameters
            
            #df_right = df_upstream.filter(regex=("upstream_total_volume_reducedmeanrunoff*|upstream_total_volume_runoff*")).clip(lower=0)
            #df_left = df_upstream.drop(list(df_right.columns), 1)
            #df_upstream_capped = df_left.merge(df_right,left_index=True,right_index=True,how="outer")         
    
            
            #sumSeries = df_upstream.sum(0)
            #sumSeries = df_upstream_capped.sum(0)
            sumSeries = df_upstream.sum(0).filter(regex=("upstream_total_volume_reducedmeanrunoff*|upstream_total_volume_runoff*")).clip(lower=0)
            for key, value in sumSeries.iteritems():
                df_out.loc[index, key] = value
            df_out.loc[index, "errorCode"] = 0
        except:
            print("error")
            df_out.loc[index, "errorCode"] = 1
            pass
    
    return df_out
    
    
def addDownstream2(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    df_out = pd.DataFrame(index=df_part_temp2.index)
    
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        try:
            upstreamCatchments = df_part_temp2.loc[index, "Downstream_PFAF_IDs"]
            upstreamCatchments = ast.literal_eval(upstreamCatchments)
            df_upstream = df_full_temp[df_full_temp.index.isin(upstreamCatchments)]
            # selecting columns based on regular expression
            df_upstream = df_upstream.filter(regex=("^total_(area|volume_(P|I))*"))
            df_upstream = df_upstream.add_prefix("downstream_")     
            
            sumSeries = df_upstream.sum(0)
            for key, value in sumSeries.iteritems():
                df_out.loc[index, key] = value
            df_out.loc[index, "errorCode"] = 0
        except:
            print("error")
            df_out.loc[index, "errorCode"] = 1
            pass
    
    return df_out


def addBasin2(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    #df_out = df_part_temp2.copy()
    df_out = pd.DataFrame(index=df_part_temp2.index)
    
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        try:
            upstreamCatchments = df_part_temp2.loc[index, "Basin_PFAF_IDs"]
            upstreamCatchments = ast.literal_eval(upstreamCatchments)
            df_upstream = df_full_temp[df_full_temp.index.isin(upstreamCatchments)]
            # selecting columns based on regular expression
            df_upstream = df_upstream.filter(regex=("^total_(area|volume_(P|I))"))
            df_upstream = df_upstream.add_prefix("basin_")
            sumSeries = df_upstream.sum(0)
            for key, value in sumSeries.iteritems():
                df_out.loc[index, key] = value
            df_out.loc[index, "errorCode"] = 0
        except:
            print("error")
            df_out.loc[index, "errorCode"] = 1
            pass
    
    return df_out

## Script

In [15]:
demandTypes = ["PDom","PInd","IrrLinear","PLiv"]
useTypes = ["WW","WN"]
temporalResolutions = ["year","month"]
years = [2014]

In [16]:
dfDemand = pd.DataFrame(index=df_complete.index)
for temporalResolution in temporalResolutions:
    for useType in useTypes:
        for year in years:
            if temporalResolution == "year":
                month = 12
                print(useType,temporalResolution,year,month)
                dfDemand = calculateTotalDemand(useType,temporalResolution,year,month)
            else:
                for month in range(1,13):
                    print(useType,temporalResolution,year,month)
                    dfDemand = calculateTotalDemand(useType,temporalResolution,year,month)          

WW year 2014 12
WN year 2014 12
WW month 2014 1
WW month 2014 2
WW month 2014 3
WW month 2014 4
WW month 2014 5
WW month 2014 6
WW month 2014 7
WW month 2014 8
WW month 2014 9
WW month 2014 10
WW month 2014 11
WW month 2014 12
WN month 2014 1
WN month 2014 2
WN month 2014 3
WN month 2014 4
WN month 2014 5
WN month 2014 6
WN month 2014 7
WN month 2014 8
WN month 2014 9
WN month 2014 10
WN month 2014 11
WN month 2014 12


In [17]:
dfDemand.head()

,total_volume_TotWW_year_Y2014,total_volume_TotWN_year_Y2014,total_volume_TotWW_month_Y2014M01,total_volume_TotWW_month_Y2014M02,total_volume_TotWW_month_Y2014M03,total_volume_TotWW_month_Y2014M04,total_volume_TotWW_month_Y2014M05,total_volume_TotWW_month_Y2014M06,total_volume_TotWW_month_Y2014M07,total_volume_TotWW_month_Y2014M08,...,total_volume_TotWN_month_Y2014M03,total_volume_TotWN_month_Y2014M04,total_volume_TotWN_month_Y2014M05,total_volume_TotWN_month_Y2014M06,total_volume_TotWN_month_Y2014M07,total_volume_TotWN_month_Y2014M08,total_volume_TotWN_month_Y2014M09,total_volume_TotWN_month_Y2014M10,total_volume_TotWN_month_Y2014M11,total_volume_TotWN_month_Y2014M12
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
111011,6.257741e+07,2.915234e+07,5.071952e+06,5.039243e+06,5.096070e+06,5.268799e+06,5.233163e+06,5.282180e+06,5.382082e+06,5.472195e+06,...,2.372698e+06,2.448676e+06,2.443976e+06,2.468962e+06,2.509266e+06,2.543447e+06,2.498844e+06,2.432985e+06,2.380002e+06,2.353863e+06
111012,8.204290e+06,3.388488e+06,6.827566e+05,6.829166e+05,6.831846e+05,6.836583e+05,6.840569e+05,6.843250e+05,6.844042e+05,6.844215e+05,...,2.818678e+05,2.823415e+05,2.827401e+05,2.830082e+05,2.830874e+05,2.831047e+05,2.829387e+05,2.826311e+05,2.820958e+05,2.816334e+05
111013,7.569537e+06,3.111568e+06,6.301277e+05,6.302355e+05,6.304261e+05,6.307669e+05,6.310496e+05,6.312474e+05,6.313145e+05,6.313229e+05,...,2.589287e+05,2.592695e+05,2.595522e+05,2.597500e+05,2.598171e+05,2.598255e+05,2.597075e+05,2.594838e+05,2.591036e+05,2.587620e+05
111014,1.046668e+06,5.024309e+05,8.651350e+04,8.664306e+04,8.685163e+04,8.721648e+04,8.751861e+04,8.770851e+04,8.775453e+04,8.777285e+04,...,4.149858e+04,4.186342e+04,4.216556e+04,4.235545e+04,4.240148e+04,4.241980e+04,4.227204e+04,4.204755e+04,4.165468e+04,4.130191e+04
111015,1.118431e+08,4.668647e+07,9.301079e+06,9.303778e+06,9.309980e+06,9.319181e+06,9.327492e+06,9.333573e+06,9.335649e+06,9.335846e+06,...,3.881478e+06,3.889610e+06,3.896934e+06,3.902287e+06,3.904112e+06,3.904281e+06,3.900583e+06,3.894476e+06,3.886032e+06,3.877061e+06


In [18]:
df_complete = df_complete.merge(dfDemand,how="left",left_index=True,right_index=True)

In [19]:
df_complete.dtypes

HYBAS_ID2                              int64
Unnamed: 0                             int64
HYBAS_ID                               int64
NEXT_DOWN                              int64
NEXT_SINK                              int64
MAIN_BAS                               int64
DIST_SINK                            float64
DIST_MAIN                            float64
SUB_AREA                             float64
UP_AREA                              float64
PFAF_ID                                int64
ENDO                                   int64
COAST                                  int64
ORDER                                  int64
SORT                                   int64
Upstream_HYBAS_IDs                    object
Upstream_PFAF_IDs                     object
Downstream_HYBAS_IDs                  object
Downstream_PFAF_IDs                   object
NEXT_SINK_PFAF                       float64
Basin_HYBAS_IDs                       object
Basin_PFAF_IDs                        object
SUB_NAME  

## Runoff Routing  
prevent negative propagation

In [20]:
df_complete["Upstream_PFAF_IDs"] = df_complete["Upstream_PFAF_IDs"].apply(lambda x: ast.literal_eval(x))

In [21]:
df_complete["numberUpstream_PFAF_IDs"] = df_complete["Upstream_PFAF_IDs"].apply(lambda x: len(x))

In [24]:
df_out = pd.DataFrame()

In [27]:
#for numberUpstream in range(0,df_complete["numberUpstream_PFAF_IDs"].max()+1):
for numberUpstream in range(0,1):
    print(numberUpstream)
    df_filtered = df_complete.loc[df_complete['numberUpstream_PFAF_IDs'] == numberUpstream].copy()
    if numberUpstream == 0:
        temporalResolution = "year"      
        # Headwaters
        df_filtered.loc[df_filtered["numberUpstream_PFAF_IDs"]==numberUpstream,"total_volume_accumulatedRunoff_year_Y2014"] = df_filtered["total_volume_runoff_yearY2014M12"]
        df_filtered.loc[df_filtered["numberUpstream_PFAF_IDs"]==numberUpstream,"total_volume_accumulatedRunoffAvailable_year_Y2014"] = df_filtered["total_volume_runoff_yearY2014M12"] - df_filtered["total_volume_TotWN_year_Y2014"]
    
        df_filtered["total_volume_accumulatedRunoff_year_Y2014"] = df_filtered["total_volume_accumulatedRunoff_year_Y2014"].clip(lower=0)
        df_filtered["total_volume_accumulatedRunoffAvailable_year_Y2014"] = df_filtered["total_volume_accumulatedRunoffAvailable_year_Y2014"].clip(lower=0)
        
        print(df_filtered["total_volume_accumulatedRunoffAvailable_year_Y2014"].clip(lower=0).min())
    
    else:
        pass
        """
        # accumulated = accumulatedRunoff_upstream + local runoff
        # accumulated available = accumulatedRunoffAvailable_upstream + local Runoff - Local consumption
        for index, row in df_filtered.iterrows():
            upstreamCatchments = df_filtered.loc[index]["Upstream_PFAF_IDs"]
            df_upstream = df_complete[df_complete.index.isin(upstreamCatchments)]
            
            df_upstream = df_upstream.filter(regex=("^total_volume_accumulated"))           
                
            sumSeries = df_upstream.sum(0)
                                           
            for key, value in sumSeries.iteritems():
                df_out.loc[index, key] = value
            df_out.loc[index, "errorCode"] = 0
            """
    df_out = df_out.append(df_filtered)
            
        

0
0.0


In [28]:
df_filtered.head()

,HYBAS_ID2,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,...,total_volume_TotWN_month_Y2014M06,total_volume_TotWN_month_Y2014M07,total_volume_TotWN_month_Y2014M08,total_volume_TotWN_month_Y2014M09,total_volume_TotWN_month_Y2014M10,total_volume_TotWN_month_Y2014M11,total_volume_TotWN_month_Y2014M12,numberUpstream_PFAF_IDs,total_volume_accumulatedRunoff_year_Y2014,total_volume_accumulatedRunoffAvailable_year_Y2014
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
111011,1060000010,6675,1060000010,0,1060000010,1060000010,0.0,0.0,1890.8,1890.8,...,2.468962e+06,2.509266e+06,2.543447e+06,2.498844e+06,2.432985e+06,2.380002e+06,2.353863e+06,0,1.343305e+06,0.0
111012,1060000100,6676,1060000100,0,1060000100,1060000100,0.0,0.0,2925.9,2925.9,...,2.830082e+05,2.830874e+05,2.831047e+05,2.829387e+05,2.826311e+05,2.820958e+05,2.816334e+05,0,5.673874e+05,0.0
111013,1060000110,6677,1060000110,0,1060000110,1060000110,0.0,0.0,893.5,893.5,...,2.597500e+05,2.598171e+05,2.598255e+05,2.597075e+05,2.594838e+05,2.591036e+05,2.587620e+05,0,1.692542e+05,0.0
111014,1060000150,6678,1060000150,0,1060000150,1060000150,0.0,0.0,4217.3,4217.4,...,4.235545e+04,4.240148e+04,4.241980e+04,4.227204e+04,4.204755e+04,4.165468e+04,4.130191e+04,0,1.544113e-05,0.0
111015,1060000160,6679,1060000160,0,1060000160,1060000160,0.0,0.0,16638.1,16638.1,...,3.902287e+06,3.904112e+06,3.904281e+06,3.900583e+06,3.894476e+06,3.886032e+06,3.877061e+06,0,9.114805e+03,0.0


In [ ]:
df_filtered

In [ ]:
df_upstream

In [ ]:
dfOut

In [ ]:
dfOut.to_csv(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME+".csv"))

In [ ]:
mp.cpu_count()

In [ ]:
indices_full = df_complete.index.values
indices_split = np.array_split(indices_full, mp.cpu_count())

In [ ]:
indices_split[1].shape

In [ ]:
mp.log_to_stderr()

In [ ]:
logger = mp.get_logger()
logger.setLevel(logging.INFO)

In [ ]:
pool = mp.Pool(mp.cpu_count())

In [ ]:
df_upstream = pd.concat(pool.map(addUpstream2, indices_split))

In [ ]:
df_upstream.loc[155697]["upstream_total_volume_reducedmeanrunoff_year_Y1960Y2014"]

In [ ]:
df_downstream = pd.concat(pool.map(addDownstream2, indices_split))

In [ ]:
df_basin = pd.concat(pool.map(addBasin2, indices_split))

In [ ]:
pool.close()

In [ ]:
df_complete = df_complete.merge(df_upstream,how="left",left_index=True,right_index=True)

In [ ]:
df_complete = df_complete.merge(df_downstream,how="left",left_index=True,right_index=True)

In [ ]:
df_complete = df_complete.merge(df_basin,how="left",left_index=True,right_index=True)

In [ ]:
df_complete.to_pickle(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME+".pkl"))

In [ ]:
df_complete.to_csv(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME+".csv"))

In [ ]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive

In [ ]:
df_complete.head()

In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)